# SOI1010 Machine Learning II - Assignment #1
#### 2022094093 Kim Dohoon, Dept. of Data Science

The submission should include a code (both link to the colab and .py format) and a report that has answers to the questions and results. Use PyTorch (or TensorFlow/JAX). Marks will be deducted if the submission does not include the requested files. DO NOT use other libraries, such as scikit-learn/sklearn, to use a model (kNN in this case) you are supposed to implement. Using sklearn or any other third library or already built-in functions that you are asked to implement will result in 0 mark. Also, if an assignment asks you to implement some model, that means you shouldn’t use the built-in implementation from any library for that model in the first place.


### Problem #1: Multiclass Classification via k-NN on MNIST [50pts]  
MNIST is a database of handwritten digit grayscale images of size 28 × 28. You cand load MNIST dataset in PyTorch as follows:

In [65]:
import numpy as np
import torch
from torchvision import datasets
import time
from tqdm.notebook import tqdm
trainset = datasets.MNIST(root='./data', train=True, download=True)
testset = datasets.MNIST(root='./data', train=False, download=True)

In [66]:
# For prettier time output
def minsec(seconds):
  output = ''
  minute = seconds // 60
  second = seconds % 60
  if minute != 0:
    output += f'{int(minute)} minute(s) '
  output += f'{second:.2f} second(s)'
  return output

##### Data split  
You should use 6000 randomly sampled images from the training set for validation. After the data split, you can use DataLoader from PyTorch as follows:

In [67]:
# Mac environment
#device = torch.device('mps')

# Colab environment
device = torch.device('cuda')

In [68]:
# Indices for train/val splits: train_idx, valid_idx
np.random.seed(0)
val_ratio = 0.1
train_size = len(trainset)
indices = list(range(train_size))
split_idx = int(np.floor(val_ratio * train_size))
np.random.shuffle(indices)

train_idx, val_idx = indices[split_idx:], indices[:split_idx]

x_train = trainset.data[train_idx].float()/255.
y_train = trainset.targets[train_idx]

x_val = trainset.data[val_idx].float()/255.
y_val = trainset.targets[val_idx]

x_test = testset.data.float()/255.
y_test = testset.targets

# Move datasets to CUDA device
x_train = x_train.to(device)
y_train = y_train.to(device)
x_val = x_val.to(device)
y_val = y_val.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

In [8]:
print(f'''
      X
train : {x_train.shape}
val : {x_val.shape}
test : {x_test.shape}

      Y
train : {y_train.shape}
val : {y_val.shape}
test : {y_test.shape}
      ''')


      X
train : torch.Size([54000, 28, 28])
val : torch.Size([6000, 28, 28])
test : torch.Size([10000, 28, 28])

      Y
train : torch.Size([54000])
val : torch.Size([6000])
test : torch.Size([10000])
      


(a) Implement an iterative method (using for loop) to classify a single new example. Write down your observations.

In [9]:
# Choose single new example from test data
sample_idx = np.random.randint(len(x_val))
sample_data = x_val[sample_idx]
sample_label = y_val[sample_idx]

print(f'Sample label : {sample_label}')

Sample label : 4


In [10]:
# Calculate Euclidean distance
def euclidean_distance(A, B):
    dist = torch.sqrt(torch.sum((A - B) ** 2))
    return dist

# k-NN
def kNN_iter(k, x_data, y_data, target):
    distances = torch.stack([euclidean_distance(target, d) for d in tqdm(x_data, total=len(x_data))])
    _, indices = distances.sort()
    k_labels = y_data[indices[:k]]

    label, count = torch.unique(k_labels, return_counts=True)
    mode_ind = torch.argmax(count)
    mode = label[mode_ind]
    return mode

In [11]:
# Question (a)
st_time = time.time()
knn_prediction = kNN_iter(5, x_train, y_train, sample_data)
iter_time = minsec(time.time()-st_time)

print(f'''
----------------------------------------------
Label : {sample_label}
Predicted : {knn_prediction}
Running time : {iter_time}
----------------------------------------------
''')

  0%|          | 0/54000 [00:00<?, ?it/s]


----------------------------------------------
Label : 4
Predicted : 4
Running time : 3.21 second(s)
----------------------------------------------



(b) Use the broadcasting concept you learned in the laboratory session to classify a single new example. Compare against the result from (a).

In [12]:
# Calculate euclidean distance
def euclidean_distance(A, B):
    dist = torch.sqrt(torch.sum((A - B) ** 2, dim=(1,2)))
    return dist

# k-NN
def kNN(k, x_data, y_data, target):
    target = target.unsqueeze(0)
    distances = euclidean_distance(x_data, target)
    _, indices = distances.sort()
    k_labels = y_data[indices[:k]]

    label, count = torch.unique(k_labels, return_counts=True)
    mode_ind = torch.argmax(count)
    mode = label[mode_ind]
    return mode


In [13]:
# Question (b)
st_time = time.time()
knn_prediction = kNN(5, x_train, y_train, sample_data)
fin_time = time.time()

print(f'''
----------------------------------------------
Label : {sample_label}
Predicted : {knn_prediction}
Running time : {minsec(fin_time-st_time)}
----------------------------------------------
''')


----------------------------------------------
Label : 4
Predicted : 4
Running time : 0.01 second(s)
----------------------------------------------



(c) Now, implement a k-NN algorithm (starting with k=5) and its training/validation/evaluation code  
to perform multiclass classification over all digits, using the implementation from (b).  
Write down your observations.

In [14]:
# Distance
def euclidean_distance(A, B):
    dist = torch.sqrt(torch.sum((A - B) ** 2, dim=(1,2)))
    return dist

# k-NN
class kNN_Classifier:
    def __init__(self, k=5):
        self.k = k

    def train(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y

    def accuracy(self, x_input, y_input):
        correct = 0
        for x_tmp, y_tmp in tqdm(zip(x_input, y_input), total=len(x_input)):
            distance = euclidean_distance(self.train_x, x_tmp)
            _, indices = distance.sort()
            k_labels = self.train_y[indices[:self.k]]
            label, count = torch.unique(k_labels, return_counts=True)
            prediction = label[torch.argmax(count)]
            if prediction == y_tmp:
                correct += 1
        return correct / len(x_input)

In [15]:
# Train model
model = kNN_Classifier()
model.train(x_train, y_train)

# Accuracy for validation dataset
st_time = time.time()
val_accuracy = model.accuracy(x_val, y_val)
val_time = minsec(time.time()-st_time)

print(f'''
----------------------------------------------
Model Evaluation

Validation Accuracy : {val_accuracy * 100:.2f}%
Validation Running time : {val_time}
----------------------------------------------
''')

  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation

Validation Accuracy : 97.27%
Validation Running time : 23.42 second(s)
----------------------------------------------



(d) Improve the algorithm from (c)  
[Hint: Try to find the desirable distance function, which can be found by googling or going through PyTorch document].  
> https://pytorch.org/docs/stable/nn.functional.html#distance-functions

In [54]:
# Euclidean Distance (L2 Norm)
# Pairwise Distance
pdist = torch.nn.PairwiseDistance(p=2)

# k-NN
class kNN_Classifier:
    def __init__(self, k=5):
        self.k = k

    def train(self, train_x, train_y):
        self.train_x = train_x.view(train_x.shape[0], -1)
        self.train_y = train_y

    def accuracy(self, x_input, y_input):
        correct = 0
        x_input = x_input.view(x_input.shape[0], -1)
        for x_tmp, y_tmp in tqdm(zip(x_input, y_input), total=len(x_input)):
            distance = pdist(self.train_x, x_tmp)
            _, indices = distance.sort()
            k_labels = self.train_y[indices[:self.k]]
            label, count = torch.unique(k_labels, return_counts=True)
            prediction = label[torch.argmax(count)]
            if prediction == y_tmp:
                correct += 1
        return correct / len(x_input)

In [55]:
model = kNN_Classifier()
model.train(x_train, y_train)

st_time = time.time()
val_accuracy = model.accuracy(x_val, y_val)
val_time = minsec(time.time()-st_time)

print(f'''
----------------------------------------------
Model Evaluation : L2 Norm, Pairwise Distance

Validation Accuracy : {val_accuracy * 100:.2f}%
Validation Running time : {val_time}
----------------------------------------------
''')

  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : L2 Norm, Pairwise Distance

Validation Accuracy : 97.27%
Validation Running time : 23.52 second(s)
----------------------------------------------



In [56]:
# Euclidean Distance (L2 Norm)
# torch.norm
pdist = torch.nn.PairwiseDistance(p=2)

# k-NN
class kNN_Classifier:
    def __init__(self, k=5):
        self.k = k

    def train(self, train_x, train_y):
        self.train_x = train_x.view(train_x.shape[0], -1)
        self.train_y = train_y

    def accuracy(self, x_input, y_input):
        correct = 0
        x_input = x_input.view(x_input.shape[0], -1)
        for x_tmp, y_tmp in tqdm(zip(x_input, y_input), total=len(x_input)):
            distance = torch.norm((self.train_x-x_tmp), p=2, dim=1)
            _, indices = distance.sort()
            k_labels = self.train_y[indices[:self.k]]
            label, count = torch.unique(k_labels, return_counts=True)
            prediction = label[torch.argmax(count)]
            if prediction == y_tmp:
                correct += 1
        return correct / len(x_input)

In [57]:
model = kNN_Classifier()
model.train(x_train, y_train)

st_time = time.time()
val_accuracy = model.accuracy(x_val, y_val)
val_time = minsec(time.time()-st_time)

print(f'''
----------------------------------------------
Model Evaluation : L2 Norm, torch.norm

Validation Accuracy : {val_accuracy * 100:.2f}%
Validation Running time : {val_time}
----------------------------------------------
''')

  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : L2 Norm, torch.norm

Validation Accuracy : 97.27%
Validation Running time : 14.59 second(s)
----------------------------------------------



(e) What are the hyperparameters you can tune?

(f) Try at least two other options for each hyperparameter. Report the performance for each option.

In [69]:
# k-NN
class kNN_Classifier:
    def __init__(self, k=5, p=1):
        self.k = k
        self.p = p

    def train(self, train_x, train_y):
        self.train_x = train_x.view(train_x.shape[0], -1)
        self.train_y = train_y

    def accuracy(self, x_input, y_input):
        correct = 0
        x_input = x_input.view(x_input.shape[0], -1)
        for x_tmp, y_tmp in tqdm(zip(x_input, y_input), total=len(x_input)):
            distance = torch.norm((self.train_x-x_tmp), p=self.p, dim=1)
            _, indices = distance.sort()
            k_labels = self.train_y[indices[:self.k]]
            label, count = torch.unique(k_labels, return_counts=True)
            prediction = label[torch.argmax(count)]
            if prediction == y_tmp:
                correct += 1
        return correct / len(x_input)

k_list = [1,3,5,10,100,200]
p_list = [1,2,float('inf')]

history = []

for k_ in k_list:
  print('==================================================')
  for p_ in p_list:
    model = kNN_Classifier(k=k_, p=p_)
    model.train(x_train, y_train)

    st_time = time.time()
    val_accuracy = model.accuracy(x_val, y_val)
    fin_time = time.time()-st_time
    val_time = minsec(fin_time)

    print(f'''
  ----------------------------------------------
  Model Evaluation : k={k_}, p={p_}

  Validation Accuracy : {val_accuracy * 100:.2f}%
  Validation Running time : {val_time}
  ----------------------------------------------
  ''')
    history.append((k_, p_, val_accuracy, fin_time))

print('==================================================')

  0%|          | 0/6000 [00:00<?, ?it/s]


  ----------------------------------------------
  Model Evaluation : k=1, p=1

  Validation Accuracy : 96.77%
  Validation Running time : 14.60 second(s)
  ----------------------------------------------
  


  0%|          | 0/6000 [00:00<?, ?it/s]


  ----------------------------------------------
  Model Evaluation : k=1, p=2

  Validation Accuracy : 97.47%
  Validation Running time : 14.56 second(s)
  ----------------------------------------------
  


  0%|          | 0/6000 [00:00<?, ?it/s]

...


  ----------------------------------------------
  Model Evaluation : k=200, p=1

  Validation Accuracy : 91.60%
  Validation Running time : 15.66 second(s)
  ----------------------------------------------
  


  0%|          | 0/6000 [00:00<?, ?it/s]


  ----------------------------------------------
  Model Evaluation : k=200, p=2

  Validation Accuracy : 92.77%
  Validation Running time : 18.26 second(s)
  ----------------------------------------------
  


  0%|          | 0/6000 [00:00<?, ?it/s]


  ----------------------------------------------
  Model Evaluation : k=200, p=inf

  Validation Accuracy : 71.02%
  Validation Running time : 14.80 second(s)
  ----------------------------------------------
  


In [74]:
# History to markdown
md = "|k value|norm|Accuracy|Running time|\n|---|---|---|---|\n"
for hist in history:
  md += f"|{hist[0]}|{hist[1]}|{hist[2]*100:.2f}%|{minsec(hist[3])}|\n"
print(md)

|k value|norm|Accuracy|Running time|
|---|---|---|---|
|1|1|96.77%|14.60 second(s)|
|1|2|97.47%|14.56 second(s)|
|1|inf|83.08%|14.70 second(s)|
|3|1|96.93%|14.71 second(s)|
|3|2|97.55%|14.49 second(s)|
|3|inf|82.08%|14.46 second(s)|
|5|1|96.45%|14.48 second(s)|
|5|2|97.27%|14.68 second(s)|
|5|inf|82.00%|14.61 second(s)|
|10|1|96.25%|14.54 second(s)|
|10|2|97.00%|14.51 second(s)|
|10|inf|81.03%|14.48 second(s)|
|100|1|93.25%|14.47 second(s)|
|100|2|94.23%|14.47 second(s)|
|100|inf|73.22%|14.58 second(s)|
|200|1|91.60%|15.66 second(s)|
|200|2|92.77%|18.26 second(s)|
|200|inf|71.02%|14.80 second(s)|



In [80]:
# Highest accuracy
history.sort(key=lambda x: -x[2])
tmp = history[0]
print(f'''
----------------------------------------------
Highest Accuracy at : k={tmp[0]}, p={tmp[1]}

Validation Accuracy : {tmp[2] * 100:.2f}%
Validation Running time : {minsec(tmp[3])}
----------------------------------------------
''')

# Fastest time
history.sort(key=lambda x: x[3])
tmp = history[0]
print(f'''
----------------------------------------------
Fastest Running time at : k={tmp[0]}, p={tmp[1]}

Validation Accuracy : {tmp[2] * 100:.2f}%
Validation Running time : {minsec(tmp[3])}
----------------------------------------------
''')


----------------------------------------------
Highest Accuracy at : k=3, p=2

Validation Accuracy : 97.55%
Validation Running time : 14.49 second(s)
----------------------------------------------


----------------------------------------------
Fastest Running time at : k=3, p=inf

Validation Accuracy : 82.08%
Validation Running time : 14.46 second(s)
----------------------------------------------



(g) You can try more options if you want. What is the final test accuracy?

In [82]:
# Cosine Similarity
import torch.nn.functional as F

# k-NN
class kNN_Classifier:
    def __init__(self, k=5):
        self.k = k

    def train(self, train_x, train_y):
        self.train_x = train_x.view(train_x.shape[0], -1)
        self.train_y = train_y

    def accuracy(self, x_input, y_input):
        correct = 0
        x_input = x_input.view(x_input.shape[0], -1)
        for x_tmp, y_tmp in tqdm(zip(x_input, y_input), total=len(x_input)):
            distance = F.cosine_similarity(self.train_x, x_tmp)
            _, indices = distance.sort(descending=True)
            k_labels = self.train_y[indices[:self.k]]
            label, count = torch.unique(k_labels, return_counts=True)
            prediction = label[torch.argmax(count)]
            if prediction == y_tmp:
                correct += 1
        return correct / len(x_input)

In [83]:
k_list = [1,3,5,10,100,200]

history = []

for k_ in k_list:
  model = kNN_Classifier(k=k_)
  model.train(x_train, y_train)

  st_time = time.time()
  val_accuracy = model.accuracy(x_val, y_val)
  fin_time = time.time()-st_time
  val_time = minsec(fin_time)

  print(f'''
----------------------------------------------
Model Evaluation : k={k_}, Cosine Similarity

Validation Accuracy : {val_accuracy * 100:.2f}%
Validation Running time : {val_time}
----------------------------------------------
''')
  history.append((k_, val_accuracy, fin_time))

  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : k=1, Cosine Similarity

Validation Accuracy : 97.97%
Validation Running time : 55.35 second(s)
----------------------------------------------



  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : k=3, Cosine Similarity

Validation Accuracy : 97.97%
Validation Running time : 54.15 second(s)
----------------------------------------------



  0%|          | 0/6000 [00:00<?, ?it/s]

...


----------------------------------------------
Model Evaluation : k=10, Cosine Similarity

Validation Accuracy : 97.63%
Validation Running time : 53.71 second(s)
----------------------------------------------



  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : k=100, Cosine Similarity

Validation Accuracy : 95.53%
Validation Running time : 53.93 second(s)
----------------------------------------------



  0%|          | 0/6000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : k=200, Cosine Similarity

Validation Accuracy : 94.27%
Validation Running time : 53.83 second(s)
----------------------------------------------



In [85]:
# History to markdown
md = "|k value|Accuracy|Running time|\n|---|---|---|\n"
for hist in history:
  md += f"|{hist[0]}|{hist[1]*100:.2f}%|{minsec(hist[2])}|\n"
print(md)

|k value|Accuracy|Running time|
|---|---|---|
|1|97.97%|55.35 second(s)|
|3|97.97%|54.15 second(s)|
|5|98.02%|53.75 second(s)|
|10|97.63%|53.71 second(s)|
|100|95.53%|53.93 second(s)|
|200|94.27%|53.83 second(s)|



In [87]:
# Highest accuracy
history.sort(key=lambda x: -x[1])
tmp = history[0]
print(f'''
----------------------------------------------
Highest Accuracy at : k={tmp[0]}

Validation Accuracy : {tmp[1] * 100:.2f}%
Validation Running time : {minsec(tmp[2])}
----------------------------------------------
''')


----------------------------------------------
Highest Accuracy at : k=5

Validation Accuracy : 98.02%
Validation Running time : 53.75 second(s)
----------------------------------------------



In [97]:
# k-NN
class kNN_Classifier:
    def __init__(self, k=5):
        self.k = k

    def train(self, train_x, train_y):
        self.train_x = train_x.view(train_x.shape[0], -1)
        self.train_y = train_y

    def accuracy(self, x_input, y_input):
        correct = 0
        x_input = x_input.view(x_input.shape[0], -1)
        for x_tmp, y_tmp in tqdm(zip(x_input, y_input), total=len(x_input)):
            distance = F.cosine_similarity(self.train_x, x_tmp)
            _, indices = distance.sort(descending=True)
            k_labels = self.train_y[indices[:self.k]]
            label, count = torch.unique(k_labels, return_counts=True)
            prediction = label[torch.argmax(count)]
            if prediction == y_tmp:
                correct += 1
        return correct / len(x_input)

In [98]:
model = kNN_Classifier()
model.train(x_train, y_train)
st_time = time.time()
final_accuracy = model.accuracy(x_test, y_test)
final_time = time.time()-st_time

print(f'''
----------------------------------------------
Model Evaluation : k=5, Cosine Similarity

Test Accuracy : {final_accuracy * 100:.2f}%
Test Running time : {minsec(final_time)}
----------------------------------------------
''')

  0%|          | 0/10000 [00:00<?, ?it/s]


----------------------------------------------
Model Evaluation : k=5, Cosine Similarity

Test Accuracy : 97.22%
Test Running time : 1 minute(s) 29.96 second(s)
----------------------------------------------

